## Importing the libraries:


In [1]:
import pandas as pd
from typing import Union, List, Dict
import numpy as np
import openpyxl
import os
import re
import xlsxwriter

from src.python.Functions import save_df_list_to_csv_auto

## Importing the datasets:


In [2]:
# Importing dataset of balance and P&L function:
# Execution might take 10 minutes or more due to the large size of the datasets.
# Cannot make pandas read csv without errors so we use openpyxl to read xlsx files.

balance2025 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2025.xlsx')
balance2024 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2024.xlsx')
balance2023 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2023.xlsx')
balance2022 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2022.xlsx')
balance2021 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2021.xlsx')
balance2020 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2020.xlsx')

pnl2025 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2025.xlsx')
pnl2024 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2024.xlsx')
pnl2023 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2023.xlsx')
pnl2022 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2022.xlsx')
pnl2021 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2021.xlsx')
pnl2020 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2020.xlsx')

### Fallbacks for large dfs:

In [19]:

# Fallbacks for large dfs:
B2025 = balance2025.copy()
B2024 = balance2024.copy()
B2023 = balance2023.copy()
B2022 = balance2022.copy()
B2021 = balance2021.copy()
B2020 = balance2020.copy()

P2025 = pnl2025.copy()
P2024 = pnl2024.copy()
P2023 = pnl2023.copy()
P2022 = pnl2022.copy()
P2021 = pnl2021.copy()
P2020 = pnl2020.copy()

# Saving in lists for functions:
balance_list = [B2025, B2024, B2023, B2022, B2021, B2020]
pnl_list = [P2025, P2024, P2023, P2022, P2021, P2020]

## Cleaning the data with functions:
#### Removing unnecessary columns:

In [26]:
# Unnecessary column removal from list of dataframes:
def remove_mutual_unnecessary_columns(df_list):
    for df in df_list:
        remove_columns = ['ja_pavadinimas', 'obj_pav','form_pav','template_name',  'standard_name', 'form_pavadinimas','line_type_id', 'stat_pavadinimas', 'stat_pav']
        for col in remove_columns:
            if col in df.columns:
                df.drop(columns=col, inplace=True)
    return df_list

# Removing unnecessary rows from list of dataframes:
remove_mutual_unnecessary_columns(balance_list)
remove_mutual_unnecessary_columns(pnl_list)

# Renaming collumns to be the same across dataframes:
def rename_columns(df_list):
    for df in df_list:
        # Rename 'obj_kodas' to 'ja_kodas' if it exists
        if 'obj_kodas' in df.columns:
            df.rename(columns={'obj_kodas': 'ja_kodas'}, inplace=True)

        # Rename other columns if they exist
        column_mapping = {
            'nuosavas_kapitalas': 'NUOSAVAS KAPITALAS',
            'mok_sumos_ir_isipareigojimai': 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI',
            'trumpalaikis_turtas': 'TRUMPALAIKIS TURTAS',
            'ilgalaikis_turtas': 'ILGALAIKIS TURTAS',
            'pelnas_pries_apmokestinima': 'PELNAS (NUOSTOLIAI) PRIEŠ APMOKESTINIMĄ',
        'grynasis_pelnas': 'GRYNASIS PELNAS (NUOSTOLIAI)',
        'pardavimo_pajamos': 'PARDAVIMO PAJAMOS'
        }

        for old_col, new_col in column_mapping.items():
            if old_col in df.columns:
                df.rename(columns={old_col: new_col}, inplace=True)

    return df_list

rename_columns(balance_list)
rename_columns(pnl_list)




[         ja_kodas  form_kodas  stat_kodas template_id standard_id  \
 0       110003978         310           0      FS0329      IST024   
 1       110003978         310           0      FS0329      IST024   
 2       110003978         310           0      FS0329      IST024   
 3       110004884         310           0      FS0718      IST209   
 4       110004884         310           0      FS0718      IST209   
 ...           ...         ...         ...         ...         ...   
 423454  307123738         310           0      FS0329      IST024   
 423455  307193537         960           0      FS0522      IST024   
 423456  307193537         960           0      FS0522      IST024   
 423457  307438075         960           0      FS0522      IST118   
 423458  307438075         960           0      FS0522      IST118   
 
                                       line_name  reiksme beginning_date  \
 0                             PARDAVIMO PAJAMOS    97545     2024-01-01   
 1    

#### Extracting columns line_name, reiksme and ja_kodas:
This function is uneeded but if you need to view just the extracted columns, you can use it:

In [27]:
# Extracting columns line_name, reiksme and ja_kodas from dfs with all of this data:
def extract_line_name_reiksme_ja_kodas(df_list):
    """
    Extract columns 'line_name', 'reiksme', and 'ja_kodas' from DataFrames
    that contain all three columns.

    Parameters:
    -----------
    df_list : list of pandas.DataFrame
        List of DataFrames to process (will be modified in-place)

    Returns:
    --------
    list of pandas.DataFrame
        List of DataFrames containing only the three specified columns
    """
    extracted_dfs = []

    required_columns = ['line_name', 'reiksme', 'ja_kodas']

    for i, df in enumerate(df_list):
        # Check if all required columns exist in the current DataFrame
        if all(col in df.columns for col in required_columns):
            # Extract only the required columns
            extracted_df = df[required_columns].copy()
            extracted_dfs.append(extracted_df)


        else:
            missing_cols = [col for col in required_columns if col not in df.columns]
            print(f"DataFrame {i}: Missing columns {missing_cols} - skipped")

    print(f"\nExtracted {len(extracted_dfs)} out of {len(df_list)} DataFrames")
    return extracted_dfs

B_extracted = extract_line_name_reiksme_ja_kodas(balance_list)
P_extracted = extract_line_name_reiksme_ja_kodas(pnl_list)

# Renaming the extracted dfs to be more descriptive:
B_extracted_2025 = B_extracted[0]
B_extracted_2024 = B_extracted[1]

P_extracted_2025 = P_extracted[0]
P_extracted_2024 = P_extracted[1]

# New lists with extracted and renamed dfs:
B_extracted_renamed = [B_extracted_2025, B_extracted_2024]
P_extracted_renamed = [P_extracted_2025, P_extracted_2024]

DataFrame 2: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 3: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 4: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 5: Missing columns ['line_name', 'reiksme'] - skipped

Extracted 2 out of 6 DataFrames
DataFrame 2: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 3: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 4: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 5: Missing columns ['line_name', 'reiksme'] - skipped

Extracted 2 out of 6 DataFrames


#### Shifting the column data of extraced dfs so that every row is unique with new columns:

In [28]:
# Shifting the column data of extraced dfs so that every row is unique:
def pivot_dfs(df_list, aggfunc='first'):
    """
    Apply pivot transformation to multiple DataFrames with many columns.
    Only uses 'line_name', 'reiksme', 'ja_kodas' for pivoting, ignores other columns.

    Parameters:
    -----------
    df_list : list of pandas.DataFrame
        List of DataFrames to pivot (can have many additional columns)
    aggfunc : str or function, default 'first'
        Aggregation function for duplicates

    Returns:
    --------
    list of pandas.DataFrame
        List of pivoted DataFrames
    """

    def pivot_line_names_to_columns(df, aggfunc='first'):
        """
        Pivot using only the three required columns, ignoring all others.
        """
        # Select only the columns needed for pivoting
        pivot_data = df[['ja_kodas', 'line_name', 'reiksme']].copy()

        # Pivot the table
        pivoted_df = pivot_data.pivot_table(
            index='ja_kodas',
            columns='line_name',
            values='reiksme',
            aggfunc=aggfunc
        ).reset_index()

        # Reset column names and clean up the DataFrame
        pivoted_df.columns.name = None

        return pivoted_df

    pivoted_dfs = []

    for i, df in enumerate(df_list):
        try:
            # Check if required columns exist
            required_columns = ['line_name', 'reiksme', 'ja_kodas']
            if not all(col in df.columns for col in required_columns):
                missing_cols = [col for col in required_columns if col not in df.columns]
                print(f"DataFrame {i}: Missing columns {missing_cols}. Available: {list(df.columns)}")
                pivoted_dfs.append(df)
                continue

            # Show info about the DataFrame
            print(f"DataFrame {i}: Original columns: {len(df.columns)}")
            print(f"DataFrame {i}: Using 3 columns for pivoting, ignoring {len(df.columns) - 3} other columns")
            print(f"DataFrame {i}: Unique line_name values: {df['line_name'].nunique()}")
            print(f"DataFrame {i}: Unique ja_kodas values: {df['ja_kodas'].nunique()}")

            # Perform pivot (only uses the 3 required columns)
            pivoted_df = pivot_line_names_to_columns(df, aggfunc)
            pivoted_dfs.append(pivoted_df)
            print(f"DataFrame {i}: Successfully pivoted. Original shape: {df.shape}, Pivoted shape: {pivoted_df.shape}")

        except Exception as e:
            print(f"DataFrame {i}: Error during pivoting - {e}")
            pivoted_dfs.append(df)

    return pivoted_dfs


DataFrame 0: Original columns: 3
DataFrame 0: Using 3 columns for pivoting, ignoring 0 other columns
DataFrame 0: Unique line_name values: 9
DataFrame 0: Unique ja_kodas values: 146512
DataFrame 0: Successfully pivoted. Original shape: (615188, 3), Pivoted shape: (146512, 10)
DataFrame 1: Original columns: 3
DataFrame 1: Using 3 columns for pivoting, ignoring 0 other columns
DataFrame 1: Unique line_name values: 9
DataFrame 1: Unique ja_kodas values: 141315
DataFrame 1: Successfully pivoted. Original shape: (604271, 3), Pivoted shape: (141315, 10)
DataFrame 0: Original columns: 3
DataFrame 0: Using 3 columns for pivoting, ignoring 0 other columns
DataFrame 0: Unique line_name values: 8
DataFrame 0: Unique ja_kodas values: 146512
DataFrame 0: Successfully pivoted. Original shape: (423459, 3), Pivoted shape: (146512, 9)
DataFrame 1: Original columns: 3
DataFrame 1: Using 3 columns for pivoting, ignoring 0 other columns
DataFrame 1: Unique line_name values: 8
DataFrame 1: Unique ja_kodas 

In [29]:
# Pivoting the big dataframes:
def pivot_dfs_smart(df_list):
    """
    Apply pivot transformation with smart aggregation for different column types.
    """

    def pivot_line_names_to_columns_smart(df):
        """
        Pivot with intelligent aggregation based on column data types.
        """
        # Identify column types for smart aggregation
        numeric_columns = []
        string_columns = []

        for col in df.columns:
            if col in ['line_name', 'reiksme']:
                continue
            if pd.api.types.is_numeric_dtype(df[col]):
                numeric_columns.append(col)
            else:
                string_columns.append(col)

        # Create aggregation dictionary
        aggregation_dict = {'reiksme': 'first'}

        # For numeric columns, use 'first' or 'mean' depending on context
        for col in numeric_columns:
            aggregation_dict[col] = 'first'

        # For string columns, use 'first' (take the first occurrence)
        for col in string_columns:
            aggregation_dict[col] = 'first'

        # Identify index columns (all columns except line_name and reiksme)
        index_columns = [col for col in df.columns if col not in ['line_name', 'reiksme']]

        # Perform pivot
        pivoted_df = df.pivot_table(
            index=index_columns,
            columns='line_name',
            values='reiksme',
            aggfunc='first'
        ).reset_index()

        # Reset column names
        pivoted_df.columns.name = None

        return pivoted_df

    pivoted_dfs = []

    for i, df in enumerate(df_list):
        try:
            required_columns = ['line_name', 'reiksme', 'ja_kodas']
            if not all(col in df.columns for col in required_columns):
                missing_cols = [col for col in required_columns if col not in df.columns]
                print(f"DataFrame {i}: Missing columns {missing_cols}")
                pivoted_dfs.append(df)
                continue

            print(f"DataFrame {i}: Preserving {len(df.columns) - 2} columns in result")

            pivoted_df = pivot_line_names_to_columns_smart(df)
            pivoted_dfs.append(pivoted_df)
            print(f"DataFrame {i}: Successfully pivoted. Original shape: {df.shape}, Pivoted shape: {pivoted_df.shape}")

        except Exception as e:
            print(f"DataFrame {i}: Error during pivoting - {e}")
            pivoted_dfs.append(df)

    return pivoted_dfs

Bpivoted



### Checking duplicate rows in all dfs:

In [30]:
# Checking duplicate rows in all dfs and removing/extracting them:

def extract_and_remove_older_duplicates(df_list, date_column='reg_date', id_column='ja_kodas'):
    """
    Find duplicate ja_kodas rows, keep the most recent reg_date, and extract older duplicates.

    Parameters:
    -----------
    df_list : list of pandas.DataFrame
        List of DataFrames to process
    date_column : str, default 'reg_date'
        Name of the date column to determine which rows to keep
    id_column : str, default 'ja_kodas'
        Name of the ID column to check for duplicates

    Returns:
    --------
    tuple: (cleaned_dfs, extracted_dfs, summary)
        cleaned_dfs: List of DataFrames with only most recent rows kept
        extracted_dfs: List of DataFrames containing the older duplicate rows
        summary: Dictionary with statistics for each DataFrame
    """
    cleaned_dfs = []
    extracted_dfs = []
    summary = {}

    for i, df in enumerate(df_list):
        try:
            # Check if required columns exist
            if id_column not in df.columns:
                print(f"DataFrame {i}: '{id_column}' column not found. Available: {list(df.columns)}")
                cleaned_dfs.append(df)
                extracted_dfs.append(pd.DataFrame())  # Empty DataFrame for consistency
                summary[i] = {'total_rows': len(df), 'duplicates_found': 0, 'kept': len(df), 'extracted': 0}
                continue

            if date_column not in df.columns:
                print(f"DataFrame {i}: '{date_column}' column not found. Available: {list(df.columns)}")
                cleaned_dfs.append(df)
                extracted_dfs.append(pd.DataFrame())
                summary[i] = {'total_rows': len(df), 'duplicates_found': 0, 'kept': len(df), 'extracted': 0}
                continue

            # Make a copy to avoid modifying original
            df_working = df.copy()

            # Ensure date_column is datetime
            if not pd.api.types.is_datetime64_any_dtype(df_working[date_column]):
                df_working[date_column] = pd.to_datetime(df_working[date_column], errors='coerce')
                # Remove rows where date conversion failed
                invalid_dates = df_working[date_column].isna().sum()
                if invalid_dates > 0:
                    print(f"DataFrame {i}: {invalid_dates} rows with invalid dates will be treated as oldest")

            # Find duplicate ja_kodas
            duplicate_mask = df_working.duplicated(subset=[id_column], keep=False)
            duplicate_rows = df_working[duplicate_mask]

            if len(duplicate_rows) == 0:
                print(f"DataFrame {i}: No duplicate {id_column} found")
                cleaned_dfs.append(df_working)
                extracted_dfs.append(pd.DataFrame())
                summary[i] = {'total_rows': len(df), 'duplicates_found': 0, 'kept': len(df), 'extracted': 0}
                continue

            # Group by ja_kodas and find the most recent date for each
            duplicate_groups = duplicate_rows.groupby(id_column)

            # Identify rows to keep (most recent date) and extract (older dates)
            keep_indices = []
            extract_indices = []

            for ja_kodas, group in duplicate_groups:
                if len(group) > 1:
                    # Find the row with the most recent date
                    most_recent_idx = group[date_column].idxmax()
                    keep_indices.append(most_recent_idx)

                    # Extract all other rows (older dates)
                    older_indices = group.index[group.index != most_recent_idx].tolist()
                    extract_indices.extend(older_indices)

            # Create DataFrames
            keep_df = df_working[~df_working.index.isin(extract_indices)]  # All non-extracted rows
            extract_df = df_working[df_working.index.isin(extract_indices)]  # Only extracted rows

            # Also include non-duplicate rows in keep_df
            non_duplicate_rows = df_working[~duplicate_mask]
            final_keep_df = pd.concat([keep_df, non_duplicate_rows], ignore_index=True)

            cleaned_dfs.append(final_keep_df)
            extracted_dfs.append(extract_df)

            # Summary statistics
            summary[i] = {
                'total_rows': len(df),
                'duplicates_found': len(duplicate_rows),
                'unique_duplicate_ids': duplicate_rows[id_column].nunique(),
                'kept': len(final_keep_df),
                'extracted': len(extract_df),
                'example_extracted': extract_df[id_column].value_counts().head(3).to_dict() if len(extract_df) > 0 else {}
            }

            print(f"DataFrame {i}: Kept {len(final_keep_df)} rows, extracted {len(extract_df)} older duplicates")
            if len(extract_df) > 0:
                print(f"  Extracted {extract_df[id_column].nunique()} unique {id_column}s with older dates")

        except Exception as e:
            print(f"DataFrame {i}: Error processing - {e}")
            cleaned_dfs.append(df)
            extracted_dfs.append(pd.DataFrame())
            summary[i] = {'total_rows': len(df), 'duplicates_found': 0, 'kept': len(df), 'extracted': 0, 'error': str(e)}

    # Final summary
    total_extracted = sum(s['extracted'] for s in summary.values())
    total_duplicates = sum(s['duplicates_found'] for s in summary.values())

    print(f"\n=== EXTRACTION SUMMARY ===")
    print(f"Total DataFrames processed: {len(df_list)}")
    print(f"Total duplicate rows found: {total_duplicates}")
    print(f"Total older rows extracted: {total_extracted}")

    return cleaned_dfs, extracted_dfs, summary

# Balance data:
cleaned_dfs, extracted_dfs, summary = extract_and_remove_older_duplicates(



DataFrame 0: Kept 147160 rows, extracted 468676 older duplicates
  Extracted 145864 unique ja_kodass with older dates
DataFrame 1: Kept 142162 rows, extracted 462956 older duplicates
  Extracted 140468 unique ja_kodass with older dates
DataFrame 2: Kept 175078 rows, extracted 152850 older duplicates
  Extracted 128160 unique ja_kodass with older dates
DataFrame 3: Kept 227434 rows, extracted 21325 older duplicates
  Extracted 10710 unique ja_kodass with older dates
DataFrame 4: Kept 204568 rows, extracted 14073 older duplicates
  Extracted 8982 unique ja_kodass with older dates
DataFrame 5: Kept 187052 rows, extracted 9965 older duplicates
  Extracted 5652 unique ja_kodass with older dates

=== EXTRACTION SUMMARY ===
Total DataFrames processed: 6
Total duplicate rows found: 1569681
Total older rows extracted: 1129845


## Adding PnL data to balance data:


In [31]:
def merge_pnl_and_balances(df_list1, df_list2, how='inner'):
    """
    Merge two lists of DataFrames on ja_kodas column with comprehensive diagnostics and validation.
    Each DataFrame from list1 is merged with corresponding DataFrame from list2.

    Parameters:
    -----------
    df_list1, df_list2 : list of pandas.DataFrame
        Lists of DataFrames to merge
    how : str, default 'inner'
        Type of merge: 'inner', 'left', 'right', 'outer'

    Returns:
    --------
    list of pandas.DataFrame
        List of merged DataFrames
    """
    if len(df_list1) != len(df_list2):
        print(f"Warning: List lengths differ - list1: {len(df_list1)}, list2: {len(df_list2)}")
        # Use the minimum length to avoid index errors
        min_length = min(len(df_list1), len(df_list2))
        df_list1 = df_list1[:min_length]
        df_list2 = df_list2[:min_length]

    merged_dfs = []

    for i, (df1, df2) in enumerate(zip(df_list1, df_list2)):
        try:
            # Check if ja_kodas exists in both DataFrames
            if 'ja_kodas' not in df1.columns:
                print(f"Pair {i}: ja_kodas not found in first DataFrame, skipping")
                continue
            if 'ja_kodas' not in df2.columns:
                print(f"Pair {i}: ja_kodas not found in second DataFrame, skipping")
                continue

            # Create copies to avoid modifying originals
            df1_clean = df1.copy()
            df2_clean = df2.copy()

            # Validate ja_kodas data types and convert if necessary
            if df1_clean['ja_kodas'].dtype != df2_clean['ja_kodas'].dtype:
                print(f"Pair {i}: ja_kodas data types differ - converting both to string")
                df1_clean['ja_kodas'] = df1_clean['ja_kodas'].astype(str)
                df2_clean['ja_kodas'] = df2_clean['ja_kodas'].astype(str)

            # Check for duplicate ja_kodas within each DataFrame
            df1_duplicates = df1_clean.duplicated(subset=['ja_kodas']).sum()
            df2_duplicates = df2_clean.duplicated(subset=['ja_kodas']).sum()

            if df1_duplicates > 0:
                print(f"Pair {i}: WARNING - {df1_duplicates} duplicate ja_kodas found in first DataFrame")
                # Keep first occurrence of duplicates
                df1_clean = df1_clean.drop_duplicates(subset=['ja_kodas'], keep='first')

            if df2_duplicates > 0:
                print(f"Pair {i}: WARNING - {df2_duplicates} duplicate ja_kodas found in second DataFrame")
                # Keep first occurrence of duplicates
                df2_clean = df2_clean.drop_duplicates(subset=['ja_kodas'], keep='first')

            # Check for NaN values in ja_kodas
            df1_nan = df1_clean['ja_kodas'].isna().sum()
            df2_nan = df2_clean['ja_kodas'].isna().sum()

            if df1_nan > 0:
                print(f"Pair {i}: WARNING - {df1_nan} NaN values in ja_kodas (first DataFrame), removing")
                df1_clean = df1_clean.dropna(subset=['ja_kodas'])

            if df2_nan > 0:
                print(f"Pair {i}: WARNING - {df2_nan} NaN values in ja_kodas (second DataFrame), removing")
                df2_clean = df2_clean.dropna(subset=['ja_kodas'])

            # Pre-merge diagnostics
            df1_unique = df1_clean['ja_kodas'].nunique()
            df2_unique = df2_clean['ja_kodas'].nunique()
            common_ja_kodas = set(df1_clean['ja_kodas']) & set(df2_clean['ja_kodas'])
            common_count = len(common_ja_kodas)

            print(f"\n--- Pair {i} Merge Diagnostics ---")
            print(f"DF1: {len(df1_clean)} rows, {df1_unique} unique ja_kodas")
            print(f"DF2: {len(df2_clean)} rows, {df2_unique} unique ja_kodas")
            print(f"Common ja_kodas: {common_count}")
            print(f"Merge type: {how}")

            # Calculate expected result sizes
            if how == 'inner':
                expected_rows = common_count
            elif how == 'left':
                expected_rows = len(df1_clean)
            elif how == 'right':
                expected_rows = len(df2_clean)
            else:  # outer
                expected_rows = len(df1_clean) + len(df2_clean) - common_count

            print(f"Expected result rows: {expected_rows}")

            # Perform the merge with indicator to track sources
            merged_df = pd.merge(
                df1_clean,
                df2_clean,
                on='ja_kodas',
                how=how,
                suffixes=('_pnl', '_balance'),
                indicator=True  # Add merge indicator column
            )

            # Post-merge diagnostics
            actual_rows = len(merged_df)
            merge_stats = merged_df['_merge'].value_counts()

            print(f"Actual result rows: {actual_rows}")
            print(f"Merge composition: {merge_stats.to_dict()}")

            if actual_rows != expected_rows:
                print(f"WARNING: Expected {expected_rows} rows but got {actual_rows} rows")

            # Remove the indicator column
            merged_df = merged_df.drop('_merge', axis=1)

            # Check for overlapping column names (besides ja_kodas)
            overlapping_cols = set(df1_clean.columns) & set(df2_clean.columns) - {'ja_kodas'}
            if overlapping_cols:
                print(f"Overlapping columns (received suffixes): {list(overlapping_cols)}")

            print(f"Pair {i}: Successfully merged. Shapes: {df1.shape} + {df2.shape} -> {merged_df.shape}")

            merged_dfs.append(merged_df)

        except Exception as e:
            print(f"Pair {i}: Error during merge - {e}")
            print(f"Pair {i}: DF1 columns: {list(df1.columns) if 'df1' in locals() else 'N/A'}")
            print(f"Pair {i}: DF2 columns: {list(df2.columns) if 'df2' in locals() else 'N/A'}")
            # Keep both original DataFrames if merge fails
            merged_dfs.extend([df1, df2])

    # Final summary
    print(f"\n=== MERGE SUMMARY ===")
    print(f"Successfully processed: {len(merged_dfs)} DataFrames")
    print(f"Total input pairs: {min(len(df_list1), len(df_list2))}")

    return merged_dfs